In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import r2_score
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error
import plotly.express as px
from sklearn.metrics import accuracy_score
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import TrainingArguments, Trainer
######
from sklearn.linear_model import LogisticRegression  # pip install scikit-learn
import pickle
import time
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.dummy import DummyRegressor
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.impute import SimpleImputer
from sklearn import set_config
from sklearn.dummy import DummyClassifier
from sklearn.metrics import ConfusionMatrixDisplay
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.naive_bayes import MultinomialNB
####
import nltk
import requests
import numpy as np
import pandas as pd
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn import set_config
set_config(display='diagram')
from sklearn.impute import SimpleImputer
pd.options.display.max_columns = 999
set_config(display='diagram')
#####
import streamlit as st
from PIL import Image
import numpy as np
import pickle
import time
import json
#import torch
from collections import Counter
import re
import string
import nltk
from nltk.corpus import stopwords
import nltk
import ssl
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#reading in rthe training set rel 3 documents
df = pd.read_excel('/Users/akamdem/Downloads/asap_aes2/training_set_rel3.xlsx')
df.head(2)


,essay_id,essay_set,essay,rater1_domain1,rater2_domain1,rater3_domain1,domain1_score,rater1_domain2,rater2_domain2,domain2_score,rater1_trait1,rater1_trait2,rater1_trait3,rater1_trait4,rater1_trait5,rater1_trait6,rater2_trait1,rater2_trait2,rater2_trait3,rater2_trait4,rater2_trait5,rater2_trait6,rater3_trait1,rater3_trait2,rater3_trait3,rater3_trait4,rater3_trait5,rater3_trait6
0,1,1,"Dear local newspaper, I think effects computer...",4.0,4.0,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",5.0,4.0,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
threshold = 10
df.dropna(thresh=len(df) - threshold, axis=1, inplace=True)
df.fillna(0, inplace=True)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12978 entries, 0 to 12977
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   essay_id        12978 non-null  int64  
 1   essay_set       12978 non-null  int64  
 2   essay           12978 non-null  object 
 3   rater1_domain1  12978 non-null  float64
 4   rater2_domain1  12978 non-null  float64
 5   domain1_score   12978 non-null  float64
dtypes: float64(3), int64(2), object(1)
memory usage: 608.5+ KB


In [6]:
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download()

2023-11-19 09:01:44.692 Python[56018:22597017] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [7]:
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     /Users/akamdem/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /Users/akamdem/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /Users/akamdem/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /Users/akamdem/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /Users/akamdem/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading pa

True

In [10]:
# Convert text to lowercase
df.essay = df.essay.str.lower()

In [11]:
# Remove punctuation
df.essay = df.essay.str.translate(str.maketrans('', '', string.punctuation))

In [12]:
tokenized_words = df.essay.apply(word_tokenize)
tokenized_sentences = df.essay.apply(sent_tokenize)

In [13]:
tokenized_sentences

0        [dear local newspaper i think effects computer...
1        [dear caps1 caps2 i believe that using compute...
2        [dear caps1 caps2 caps3 more and more people u...
3        [dear local newspaper caps1 i have found that ...
4        [dear location1 i know having computers has a ...
                               ...                        
12973    [ in most stories mothers and daughters are ei...
12974    [ i never understood the meaning laughter is t...
12975    [when you laugh is caps5 out of habit or is ca...
12976    [                               trippin on fen...
12977    [ many people believe that laughter can improv...
Name: essay, Length: 12978, dtype: object

In [14]:
# Remove stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
df.essay = df.essay.apply(
    lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/akamdem/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
df.head(2)

,essay_id,essay_set,essay,rater1_domain1,rater2_domain1,domain1_score
0,1,1,dear local newspaper think effects computers p...,4.0,4.0,8.0
1,2,1,dear caps1 caps2 believe using computers benef...,5.0,4.0,9.0


In [16]:
#tokenized_words = [word for word in tokenized_words if word not in stop_words]
#tokenized_sentences = [word for word in tokenized_sentences if word not in stop_words]
text_input = df.essay#["This is a positive review.", "This is a negative review.", "Neutral review."]
scores = df.domain1_score #[1, 0, 1]  # Example scores (1 for positive, 0 for negative)

In [17]:
# Step 1: Text Vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=1000)  # You can adjust max_features
X = tfidf_vectorizer.fit_transform(text_input)
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, scores, test_size=0.2, random_state=42)

In [18]:
#X_train, X_test, y_train, y_test = train_test_split(df['essay'], df['domain1_score'], test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, scores, test_size=0.2, random_state=42)

In [19]:
model = MultinomialNB()
model.fit(X_train, y_train)

MultinomialNB()

In [20]:
predicted_scores = model.predict(X_test)

In [21]:
accuracy = metrics.accuracy_score(y_test, predicted_scores)
precision = metrics.precision_score(y_test, predicted_scores, average='weighted')
recall = metrics.recall_score(y_test, predicted_scores, average='weighted')
f1_score = metrics.f1_score(y_test, predicted_scores, average='weighted')

# Print the evaluation scores
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1-Score: {f1_score}')

Accuracy: 0.35939907550077044
Precision: 0.32438895451943855
Recall: 0.35939907550077044
F1-Score: 0.3090308996360686


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
